In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ion_toolkit import Ion

In [11]:
ion = Ion("Ba", 138)

In [15]:
for energy_level in ion.energy_levels:
    print(energy_level)
    for zeeman_level in energy_level.zeeman_levels:
        print(zeeman_level)


FineStructure(energy=0.0 THz, n=6, I=0, L=0, J=0.5)
FineStructureZeemanLevel(energy=0.0 THz, n=6, I=0, L=0, J=0.5, m_J=-0.5)
FineStructureZeemanLevel(energy=0.0 THz, n=6, I=0, L=0, J=0.5, m_J=0.5)
FineStructure(energy=607.4263 THz, n=6, I=0, L=1, J=0.5)
FineStructureZeemanLevel(energy=607.4263 THz, n=6, I=0, L=1, J=0.5, m_J=-0.5)
FineStructureZeemanLevel(energy=607.4263 THz, n=6, I=0, L=1, J=0.5, m_J=0.5)
FineStructure(energy=658.1165 THz, n=6, I=0, L=1, J=1.5)
FineStructureZeemanLevel(energy=658.1165 THz, n=6, I=0, L=1, J=1.5, m_J=-1.5)
FineStructureZeemanLevel(energy=658.1165 THz, n=6, I=0, L=1, J=1.5, m_J=-0.5)
FineStructureZeemanLevel(energy=658.1165 THz, n=6, I=0, L=1, J=1.5, m_J=0.5)
FineStructureZeemanLevel(energy=658.1165 THz, n=6, I=0, L=1, J=1.5, m_J=1.5)
FineStructure(energy=146.1144 THz, n=5, I=0, L=2, J=1.5)
FineStructureZeemanLevel(energy=146.1144 THz, n=5, I=0, L=2, J=1.5, m_J=-1.5)
FineStructureZeemanLevel(energy=146.1144 THz, n=5, I=0, L=2, J=1.5, m_J=-0.5)
FineStructu